In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
from sklearn.model_selection import GridSearchCV, train_test_split
from xgboost import XGBRegressor

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [0]:
import math
from scipy import stats

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
link='https://drive.google.com/open?id=1pjUCoAixcdmnuLcSEscEgUHV7KGGiTU-'


     |████████████████████████████████| 993kB 2.7MB/s 


In [0]:
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

1pjUCoAixcdmnuLcSEscEgUHV7KGGiTU-


In [0]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('ML Data Scientist Case Study Data.csv')  

### First : we import the data

In [0]:
data_raw=pd.read_csv('ML Data Scientist Case Study Data.csv' , delimiter=';')

### We run some describtives , we check the structure of data , missing or null values , and how data is stored in each column 

In [0]:
data_raw.head()

,row_num,locale,day_of_week,hour_of_day,agent_id,entry_page,path_id_set,traffic_type,session_durantion,hits
0,988681,L6,Monday,17,1,2111,31672;0,6,7037,\N
1,988680,L2,Thursday,22,10,2113,31965;0,2,49,14
2,988679,L4,Saturday,21,2,2100,0;78464,1,1892,14
3,988678,L3,Saturday,19,8,2113,51462,6,0,1
4,988677,L2,Tuesday,6,10,2116,31931;0,1,2,3


#### we notice that hits column has some missing values coded as '\N' so we can deal with it as target of our prediction

In [0]:
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 988681 entries, 0 to 988680
Data columns (total 10 columns):
row_num              988681 non-null int64
locale               988681 non-null object
day_of_week          988681 non-null object
hour_of_day          988681 non-null int64
agent_id             988681 non-null int64
entry_page           988681 non-null int64
path_id_set          983792 non-null object
traffic_type         988681 non-null int64
session_durantion    988681 non-null object
hits                 988681 non-null object
dtypes: int64(5), object(5)
memory usage: 75.4+ MB


#### Let's check percentage of 'hits' values needed to be predicted 

In [0]:
np.round(len(data_raw.loc[data_raw['hits'] =='\\N'])*100/len(data_raw),2)

37.37

#### we make an indicator to use later in splitting data to dataset to use in building the model and dataset to predict missing hits

In [0]:
data_raw['predicting']=(data_raw['hits'] !='\\N')*1

In [0]:
data_raw.head()

,row_num,locale,day_of_week,hour_of_day,agent_id,entry_page,path_id_set,traffic_type,session_durantion,hits,predicting
0,988681,L6,Monday,17,1,2111,31672;0,6,7037,\N,0
1,988680,L2,Thursday,22,10,2113,31965;0,2,49,14,1
2,988679,L4,Saturday,21,2,2100,0;78464,1,1892,14,1
3,988678,L3,Saturday,19,8,2113,51462,6,0,1,1
4,988677,L2,Tuesday,6,10,2116,31931;0,1,2,3,1


#### for better processing for the data, we transfer path_id_set into a list

In [0]:
data_raw['path_id_set_new']=data_raw['path_id_set'].astype('str').str.split(';')

In [0]:
data_raw.head(10)

,row_num,locale,day_of_week,hour_of_day,agent_id,entry_page,path_id_set,traffic_type,session_durantion,hits,predicting,path_id_set_new
0,988681,L6,Monday,17,1,2111,31672;0,6,7037,\N,0,"[31672, 0]"
1,988680,L2,Thursday,22,10,2113,31965;0,2,49,14,1,"[31965, 0]"
2,988679,L4,Saturday,21,2,2100,0;78464,1,1892,14,1,"[0, 78464]"
3,988678,L3,Saturday,19,8,2113,51462,6,0,1,1,[51462]
4,988677,L2,Tuesday,6,10,2116,31931;0,1,2,3,1,"[31931, 0]"
5,988676,L3,Monday,1,8,2100,0,1,0,2,1,[0]
6,988675,L2,Friday,20,10,2166,0,1,0,2,1,[0]
7,988674,L3,Sunday,3,1,2113,34302;0,6,2,3,1,"[34302, 0]"
8,988673,L3,Sunday,1,10,2111,34227;0,6,4,3,1,"[34227, 0]"
9,988672,L6,Sunday,5,9,2113,42172;0;38715,2,724,45,1,"[42172, 0, 38715]"


#### We then take the frequencies for several variables that may need to use to predict missing hits



In [0]:
f=['locale'	,'day_of_week','hour_of_day','agent_id','entry_page','traffic_type','session_durantion']
for i in f :
  print(i)
  print(data_raw[i].value_counts())
  print('  ')
  print('__________________')
  


locale
L3    352454
L6    178430
L2    172904
L4    131271
L5    112054
L1     41568
Name: locale, dtype: int64
  
__________________
day_of_week
Monday       158685
Tuesday      155001
Wednesday    147099
Sunday       144330
Thursday     136476
Friday       127740
Saturday     119350
Name: day_of_week, dtype: int64
  
__________________
hour_of_day
19    64558
20    62676
18    61810
17    55743
21    55437
16    54602
15    54139
14    51718
13    47960
22    44303
12    44062
0     41263
11    39462
23    38038
1     36206
10    34768
2     31961
9     31115
8     27907
3     25421
7     24394
6     21235
4     20638
5     19265
Name: hour_of_day, dtype: int64
  
__________________
agent_id
10    287869
9     194367
1     184228
8      94680
6      59907
2      57077
13     48169
7      26000
14     24743
0       5359
12      3694
11      1986
4        222
5        159
3        121
15       100
Name: agent_id, dtype: int64
  
__________________
entry_page
2113    333163
2116    1959

#### we notice that all categorical variables have limited values to take , or 'like : entry page ' we find that the first 10 or 15 values have the most frequencies 

#### then we make sure that each column is stored in the proper data type

In [0]:
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 988681 entries, 0 to 988680
Data columns (total 12 columns):
row_num              988681 non-null int64
locale               988681 non-null object
day_of_week          988681 non-null object
hour_of_day          988681 non-null int64
agent_id             988681 non-null int64
entry_page           988681 non-null int64
path_id_set          983792 non-null object
traffic_type         988681 non-null int64
session_durantion    988681 non-null object
hits                 988681 non-null object
predicting           988681 non-null int64
path_id_set_new      988681 non-null object
dtypes: int64(6), object(6)
memory usage: 90.5+ MB




#### we notice that session duration is a numeric column despite it is stored as object 

#### to be able to store it properly, we remove the symbol that replaces missing values'/N' with zero , since this is the most logical value for missing session duration , also it is the mode as clear from frequencies table above 

In [0]:
data_raw['session_durantion']=data_raw['session_durantion'].str.replace('N','0')

data_raw['session_durantion']=data_raw['session_durantion'].str.replace('\\','0')


In [0]:
data_raw['session_durantion']=data_raw['session_durantion'].astype('int')

#### doing the similar with hits , anyway our target from the whole case study is ti replace the'\N' then we for now replace missing values in it with zeros to be able to trasfer it into integers till we predict them in the following steps

In [0]:
data_raw['hits']=data_raw['hits'].str.replace('N','0')

data_raw['hits']=data_raw['hits'].str.replace('\\','0')


In [0]:
data_raw['hits']=data_raw['hits'].astype('int')

#### on the other hand, 'hour_of_the_day' , 'agent_id','entry_page', and 'traffic_type' despite represented as numeric but their magnitude is string , categorical values , so we represent them as categorical variables.

In [0]:
data_raw['hour_of_day']=data_raw['hour_of_day'].astype('str')

In [0]:
data_raw['agent_id']=data_raw['agent_id'].astype('str')

In [0]:
data_raw['entry_page']=data_raw['entry_page'].astype('str')

In [0]:
data_raw['traffic_type']=data_raw['traffic_type'].astype('str')

#### one of the characteristics in the data that may help to detect effective factors that affect number of hits visited pages , we take the top 4 visited pages , then make an indicator for each

#### first we detect the top 4 visited pages 

In [0]:
f=[]
x=data_raw['path_id_set'].astype('str').str.split(';')
for i in x :
    for j in i :
      f.append(j)
    


In [0]:
x=pd.DataFrame(pd.Series(f).value_counts(), columns=['freq'] )


In [0]:
x=x.loc[x['freq']>= 1000].head(4)
x.index.name='path_id'

In [0]:
x

,freq
path_id,
0,898419
38715,21011
34741,16019
34812,14971


In [0]:
x.index

Index(['0', '38715', '34741', '34812'], dtype='object', name='path_id')

#### then we compute number of visited pages per session

In [0]:
data_raw['number_of_pages']=len(data_raw['path_id_set_new'])

In [0]:
f=[]
for i in data_raw['path_id_set_new'].index:
    j=len(data_raw['path_id_set_new'][i])
    f.append(j)

In [0]:
data_raw['number_of_visited_pages'] = f

#### we compute correlation coeffecient to make sure that numeric variables can help to predict number of hits 

In [0]:
data_raw.loc[data_raw['predicting']==1,['hits','number_of_visited_pages' ,'session_durantion']].corr()['hits']

hits                       1.000000
number_of_visited_pages    0.403462
session_durantion          0.245583
Name: hits, dtype: float64

#### we notice that 'number_of_visited_pages' has moderate positive correlation with hits , and session_durantion has a weak correlation 

#### then in the following loop , we make a binary column for each of the top 4 visited pages to help to predict hits

In [0]:
for x1 in x.index :
  innerlist = []
  for i in data_raw['path_id_set_new'].index:
    j1=pd.Series(data_raw['path_id_set_new'][i]).isin([x1]).sum() > 0
    innerlist.append(j1)
  data_raw[x1]=innerlist





In [0]:
data_raw.head()

,row_num,locale,day_of_week,hour_of_day,agent_id,entry_page,path_id_set,traffic_type,session_durantion,hits,predicting,path_id_set_new,number_of_pages,number_of_visited_pages,0,38715,34741,34812
0,988681,L6,Monday,17,1,2111,31672;0,6,7037,0,0,"[31672, 0]",988681,2,True,False,False,False
1,988680,L2,Thursday,22,10,2113,31965;0,2,49,14,1,"[31965, 0]",988681,2,True,False,False,False
2,988679,L4,Saturday,21,2,2100,0;78464,1,1892,14,1,"[0, 78464]",988681,2,True,False,False,False
3,988678,L3,Saturday,19,8,2113,51462,6,0,1,1,[51462],988681,1,False,False,False,False
4,988677,L2,Tuesday,6,10,2116,31931;0,1,2,3,1,"[31931, 0]",988681,2,True,False,False,False


#### then we also we need to make indicator columns for entry pages since they may help in predicting hits , but since we have many pages , we only make an indicator for pages that were entry pages for more than 1000 sessions 

In [0]:
f=list(data_raw['entry_page'].value_counts()[(data_raw['entry_page'].value_counts() > 1000)].index)
f

['2113',
 '2116',
 '2100',
 '2111',
 '2114',
 '2700',
 '2115',
 '8001',
 '2226',
 '2316',
 '2227',
 '2142',
 '2391',
 '2704']

#### First we define a new column that takes 0 if the entry page not one of the pages chosen above 

In [0]:
data_raw['entry_page1'] = 0 

In [0]:
data_raw['entry_page1'] = [x if x in f  else 0  for x in data_raw['entry_page']]

In [0]:
  data_raw['entry_page1'].value_counts()

2113    333163
2116    195904
2100    178912
2111     83687
2114     74050
2700     58330
2115     16733
8001     16309
0        10756
2226      9840
2316      3332
2227      3024
2142      1886
2391      1544
2704      1211
Name: entry_page1, dtype: int64

#### then we make dummy variable for each of the columns , entry_page (after modification) , traffic_type , locale , and day of week 

In [0]:
all_data=pd.get_dummies(data_raw.loc[:,['locale','day_of_week','traffic_type','entry_page1']])

In [0]:
all_data.head()

,locale_L1,locale_L2,locale_L3,locale_L4,locale_L5,locale_L6,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday,traffic_type_1,traffic_type_10,traffic_type_2,traffic_type_3,traffic_type_4,traffic_type_6,traffic_type_7,entry_page1_0,entry_page1_2100,entry_page1_2111,entry_page1_2113,entry_page1_2114,entry_page1_2115,entry_page1_2116,entry_page1_2142,entry_page1_2226,entry_page1_2227,entry_page1_2316,entry_page1_2391,entry_page1_2700,entry_page1_2704,entry_page1_8001
0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [0]:
data_raw.count()

row_num                    988681
locale                     988681
day_of_week                988681
hour_of_day                988681
agent_id                   988681
entry_page                 988681
path_id_set                983792
traffic_type               988681
session_durantion          988681
hits                       988681
predicting                 988681
path_id_set_new            988681
number_of_pages            988681
number_of_visited_pages    988681
0                          988681
38715                      988681
34741                      988681
34812                      988681
entry_page1                988681
dtype: int64

#### then we merge the newly generated columns to the data 

In [0]:
all_data_1=pd.concat([data_raw,all_data],axis=1)

In [0]:
all_data_1.head()

,row_num,locale,day_of_week,hour_of_day,agent_id,entry_page,path_id_set,traffic_type,session_durantion,hits,predicting,path_id_set_new,number_of_pages,number_of_visited_pages,0,38715,34741,34812,entry_page1,locale_L1,locale_L2,locale_L3,locale_L4,locale_L5,locale_L6,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday,traffic_type_1,traffic_type_10,traffic_type_2,traffic_type_3,traffic_type_4,traffic_type_6,traffic_type_7,entry_page1_0,entry_page1_2100,entry_page1_2111,entry_page1_2113,entry_page1_2114,entry_page1_2115,entry_page1_2116,entry_page1_2142,entry_page1_2226,entry_page1_2227,entry_page1_2316,entry_page1_2391,entry_page1_2700,entry_page1_2704,entry_page1_8001
0,988681,L6,Monday,17,1,2111,31672;0,6,7037,0,0,"[31672, 0]",988681,2,True,False,False,False,2111,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,988680,L2,Thursday,22,10,2113,31965;0,2,49,14,1,"[31965, 0]",988681,2,True,False,False,False,2113,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,988679,L4,Saturday,21,2,2100,0;78464,1,1892,14,1,"[0, 78464]",988681,2,True,False,False,False,2100,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,988678,L3,Saturday,19,8,2113,51462,6,0,1,1,[51462],988681,1,False,False,False,False,2113,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,988677,L2,Tuesday,6,10,2116,31931;0,1,2,3,1,"[31931, 0]",988681,2,True,False,False,False,2116,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


#### then we drop columns that were transferred to binary columns and columns that were stored in another type (i.e path_id_set)

#### then we divide the data to training data (the data in which hits is not missing) , and test data (in which hits is missing) also we have an important notice

#### we have columns like day_of_the_week  , locale and traffic_type , if we take all binary variables in analysis we may have collinearity so we exclude the binary column of l'locale_L1','traffic_type_1','entry_page1_0'

In [0]:
all_data_1=all_data_1.drop(columns=['path_id_set','number_of_pages','locale','day_of_week','traffic_type','entry_page','entry_page1','day_of_week_Friday','locale_L1','traffic_type_1','entry_page1_0'])

In [0]:
all_data_1.head()

,row_num,hour_of_day,agent_id,session_durantion,hits,predicting,path_id_set_new,number_of_visited_pages,0,38715,34741,34812,locale_L2,locale_L3,locale_L4,locale_L5,locale_L6,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday,traffic_type_10,traffic_type_2,traffic_type_3,traffic_type_4,traffic_type_6,traffic_type_7,entry_page1_2100,entry_page1_2111,entry_page1_2113,entry_page1_2114,entry_page1_2115,entry_page1_2116,entry_page1_2142,entry_page1_2226,entry_page1_2227,entry_page1_2316,entry_page1_2391,entry_page1_2700,entry_page1_2704,entry_page1_8001
0,988681,17,1,7037,0,0,"[31672, 0]",2,True,False,False,False,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,988680,22,10,49,14,1,"[31965, 0]",2,True,False,False,False,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,988679,21,2,1892,14,1,"[0, 78464]",2,True,False,False,False,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,988678,19,8,0,1,1,[51462],1,False,False,False,False,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,988677,6,10,2,3,1,"[31931, 0]",2,True,False,False,False,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [0]:
train = all_data_1.loc[all_data_1['predicting'] == 1,['session_durantion' , 'number_of_visited_pages' ,'0',	'38715',	'34741',	'34812',	'locale_L2',	'locale_L3',	'locale_L4',	'locale_L5',	'locale_L6',	'day_of_week_Monday',	'day_of_week_Saturday',	'day_of_week_Sunday',	'day_of_week_Thursday',	'day_of_week_Tuesday',	'day_of_week_Wednesday',	'traffic_type_10',	'traffic_type_2',	'traffic_type_3',	'traffic_type_4',	'traffic_type_6',	'traffic_type_7',	'entry_page1_2100',	'entry_page1_2111',	'entry_page1_2113',	'entry_page1_2114',	'entry_page1_2115',	'entry_page1_2116',	'entry_page1_2142',	'entry_page1_2226',	'entry_page1_2227',	'entry_page1_2316',	'entry_page1_2700',	'entry_page1_8001'
 ]]

#### also y_train is available hits , and x test is columns of final data where hits is missing

#### in x test we add row_num to make it the label for the predicted values as demanded in the case study 

In [0]:
y_train = all_data_1.loc[all_data_1['predicting'] == 1,['hits']]

In [0]:
x_test = all_data_1.loc[all_data_1['predicting'] == 0,['row_num','session_durantion' , 'number_of_visited_pages' ,'0',	'38715',	'34741',	'34812',	'locale_L2',	'locale_L3',	'locale_L4',	'locale_L5',	'locale_L6',	'day_of_week_Monday',	'day_of_week_Saturday',	'day_of_week_Sunday',	'day_of_week_Thursday',	'day_of_week_Tuesday',	'day_of_week_Wednesday',	'traffic_type_10',	'traffic_type_2',	'traffic_type_3',	'traffic_type_4',	'traffic_type_6',	'traffic_type_7',	'entry_page1_2100',	'entry_page1_2111',	'entry_page1_2113',	'entry_page1_2114',	'entry_page1_2115',	'entry_page1_2116',	'entry_page1_2142',	'entry_page1_2226',	'entry_page1_2227',	'entry_page1_2316',	'entry_page1_2700',	'entry_page1_8001'
 ]]

In [0]:
x_test.head()

,row_num,session_durantion,number_of_visited_pages,0,38715,34741,34812,locale_L2,locale_L3,locale_L4,locale_L5,locale_L6,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday,traffic_type_10,traffic_type_2,traffic_type_3,traffic_type_4,traffic_type_6,traffic_type_7,entry_page1_2100,entry_page1_2111,entry_page1_2113,entry_page1_2114,entry_page1_2115,entry_page1_2116,entry_page1_2142,entry_page1_2226,entry_page1_2227,entry_page1_2316,entry_page1_2700,entry_page1_8001
0,988681,7037,2,True,False,False,False,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
15,988666,5189,3,True,False,False,False,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
16,988665,5,2,True,False,False,False,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
17,988664,8041,3,True,False,False,False,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
18,988663,117,3,True,False,False,False,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [0]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 619235 entries, 1 to 988680
Data columns (total 35 columns):
session_durantion          619235 non-null int64
number_of_visited_pages    619235 non-null int64
0                          619235 non-null bool
38715                      619235 non-null bool
34741                      619235 non-null bool
34812                      619235 non-null bool
locale_L2                  619235 non-null uint8
locale_L3                  619235 non-null uint8
locale_L4                  619235 non-null uint8
locale_L5                  619235 non-null uint8
locale_L6                  619235 non-null uint8
day_of_week_Monday         619235 non-null uint8
day_of_week_Saturday       619235 non-null uint8
day_of_week_Sunday         619235 non-null uint8
day_of_week_Thursday       619235 non-null uint8
day_of_week_Tuesday        619235 non-null uint8
day_of_week_Wednesday      619235 non-null uint8
traffic_type_10            619235 non-null uint8
traffic_typ

In [0]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 619235 entries, 1 to 988680
Data columns (total 1 columns):
hits    619235 non-null int64
dtypes: int64(1)
memory usage: 9.4 MB


### Then we transfer to modelling

#### we use XGB regressor which proved successful and many of winning solutions in kaggle competitions depended on it 

#### we use train test split , meaning that the train set we use also to be divided to train and test datasets , we use the default of 0.75 for training data

#### to have the most satisfactory solution 

#### we got we use cross validation , and grid search to guarntee that the chosen parameters give the best result  

In [0]:
# Organize our data for training
X = train
Y = y_train
X, X_Val, Y, Y_Val = train_test_split(X, Y)

In [0]:


# A parameter grid for XGBoost
params = {'min_child_weight':[3,4], 'gamma':[i/10.0 for i in [2,3]],  'subsample':[i/10.0 for i in range(5,8)],
'colsample_bytree':[i/10.0 for i in range(5,8)], 'max_depth': [5,6]}

# Initialize XGB and GridSearch
xgb = XGBRegressor(nthread=-1) 

grid = GridSearchCV(xgb, params)
grid.fit(X, Y)

# Print the r2 score
print(r2_score(Y_Val, grid.best_estimator_.predict(X_Val))) 
# Print the r2 score
print(mean_squared_error(Y_Val, grid.best_estimator_.predict(X_Val))) 



/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


[20:22:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:22:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:23:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:23:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:23:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:24:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:24:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:24:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

#### we need to scroll the above result to the end to see that 
#### R2: 0.34293812497453036
#### MSE : 971.5537531206933

In [0]:
grid.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, gamma=0.2,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=4, missing=None, n_estimators=100,
             n_jobs=1, nthread=-1, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.7, verbosity=1)

In [0]:
# Organize our data for training
X = train
Y = y_train
X, X_Val, Y, Y_Val = train_test_split(X, Y)

In [0]:


# A parameter grid for XGBoost
params = {'min_child_weight':[4], 'gamma':[i/10.0 for i in [3]],  'subsample':[i/10.0 for i in range(6)],
'colsample_bytree':[i/10.0 for i in [5]], 'max_depth': [5] ,'learning_rate': [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]}

# Initialize XGB and GridSearch
xgb = XGBRegressor(nthread=-1) 

grid1 = GridSearchCV(xgb, params)
grid1.fit(X, Y)

# Print the r2 score
print(r2_score(Y_Val, grid1.best_estimator_.predict(X_Val))) 
# Print the mse score
print(mean_squared_error(Y_Val, grid1.best_estimator_.predict(X_Val))) 

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


[19:09:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:09:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:09:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:09:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:10:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:10:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:10:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:10:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

#### we need to scroll the above result to the end to see that 
#### R2:0.3537604997286763

#### MSE : 965.8472312459871

#### below the parameters of the best model

In [0]:
grid1.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, gamma=0.3,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=4, missing=None, n_estimators=100,
             n_jobs=1, nthread=-1, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.5, verbosity=1)

In [0]:

# A parameter grid for XGBoost
params = {'min_child_weight':range(4,6), 'gamma':[i/10.0 for i in [3,4,5]],  'subsample':[i/10.0 for i in [5]],
'colsample_bytree':[i/10.0 for i in range(3,5)], 'max_depth': [3,4,5] ,'n_estimators' : [100,500,1000]}

# Initialize XGB and GridSearch
xgb = XGBRegressor(nthread=-1) 

grid2 = GridSearchCV(xgb, params)
grid2.fit(X, Y)

# Print the r2 score
print(r2_score(Y_Val, grid2.best_estimator_.predict(X_Val))) 
# Print the r2 score
print(mean_squared_error(Y_Val, grid2.best_estimator_.predict(X_Val))) 

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


[11:54:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:54:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:54:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:55:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:55:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:56:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:57:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:58:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

ٌ#### we need to scroll the above result to the end to see that 
#### R2:0.3351687238988875

#### MSE : 963.550585807207
0


#### then we apply the best model to the test data(data in which hits is missing) indicating results with row number

In [0]:
grid2.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.4, gamma=0.3,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=4, missing=None, n_estimators=100,
             n_jobs=1, nthread=-1, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.5, verbosity=1)

### , this seems as clear from E and R2 not so satisfactory result
### But we may have needed more parameter tuning but this is what I could do with computational power of my computer. 


In [0]:
# Save the file
y_test = np.round(grid.best_estimator_.predict(x_test.iloc[:,1:]),0)
results_df = pd.DataFrame(data={'y':y_test} , index=x_test['row_num']) 

In [0]:
results_df.head()

,y
row_num,
988681,55.0
988666,79.0
988665,3.0
988664,131.0
988663,19.0


#### we make row number in a separate column 

In [0]:
results_df1=pd.DataFrame(results_df.reset_index())

In [0]:
results_df1.head()

,row_num,y
0,988681,55.0
1,988666,79.0
2,988665,3.0
3,988664,131.0
4,988663,19.0


In [0]:
results_df1.to_csv("hits_predicted.csv", index=False)
